# SAM 2 Product Extraction Demo

Extract product regions from retail ad images using SAM 2 on Snowflake SPCS.

**What this notebook does:**
1. Verifies the SAM 2 detection service is running
2. Lists ad images from `AD_INPUT_STAGE/ads/`
3. Calls `EXTRACT_PRODUCTS` to process all images at once
4. Displays extraction results and statistics
5. Shows how to access the cropped product images

**Prerequisites:**
- Upload this notebook to Snowflake (Snowsight → Projects → Notebooks)
- Select warehouse: `SAM_DEMO_WH`
- Database/Schema: `SHALION_HF_DEMO.PRODUCT_EXTRACTION`
- Ensure `SAM2_DETECTOR_SERVICE` is READY


## Setup: Import Libraries and Get Session


In [ ]:
import json
from snowflake.snowpark.context import get_active_session

# Get the active Snowpark session
session = get_active_session()
print("✓ Session connected")


## Step 1: Verify Service Status

Check that the SAM 2 detection service is running before processing.


In [ ]:
# Check service status
status = session.sql("""
    CALL SYSTEM$GET_SERVICE_STATUS('SAM2_DETECTOR_SERVICE')
""").collect()[0][0]

status_json = json.loads(status)
if status_json and status_json[0].get('status') == 'READY':
    print("✓ SAM2_DETECTOR_SERVICE is READY")
else:
    print(f"⚠ Service status: {status}")
    print("Please wait for the service to be READY before proceeding.")


## Step 2: List Available Input Images

Query the input stage to see what images are available for processing.


In [ ]:
# List images in the ads folder
images_df = session.sql("""
    SELECT 
        RELATIVE_PATH,
        SIZE AS file_size_bytes,
        LAST_MODIFIED
    FROM DIRECTORY(@AD_INPUT_STAGE)
    WHERE RELATIVE_PATH LIKE 'ads/%'
      AND (RELATIVE_PATH ILIKE '%.jpg' 
           OR RELATIVE_PATH ILIKE '%.jpeg' 
           OR RELATIVE_PATH ILIKE '%.png')
    ORDER BY RELATIVE_PATH
""")

image_count = images_df.count()
print(f"Found {image_count} images in @AD_INPUT_STAGE/ads/")
images_df.show(10)


## Step 3: Run Product Extraction

Call the `EXTRACT_PRODUCTS` stored procedure to:
1. Detect products using SAM 2 on GPU
2. Deduplicate overlapping detections
3. Crop and save product images to output stage


In [ ]:
# Run extraction on the 'ads' folder
# This processes ALL images in @AD_INPUT_STAGE/ads/
print("Starting product extraction...")
print("This may take a few minutes depending on the number of images.\n")

result = session.sql("""
    CALL EXTRACT_PRODUCTS('ads', '@AD_OUTPUT_STAGE')
""").collect()[0][0]

# Parse the JSON result
extraction_result = json.loads(result)
print("✓ Extraction complete!")


## Step 4: Display Results

Show extraction statistics including timing and throughput.


In [ ]:
print("=" * 60)
print("EXTRACTION RESULTS")
print("=" * 60)
print(f"")
print(f"📊 Summary:")
print(f"   Images processed:    {extraction_result['total_images']}")
print(f"   Products extracted:  {extraction_result['total_products']}")
print(f"   Failed crops:        {extraction_result['failed_crops']}")
print(f"")
print(f"⏱️  Timing:")
print(f"   Detection (GPU):     {extraction_result['detection_seconds']:.2f}s")
print(f"   Cropping (CPU):      {extraction_result['cropping_seconds']:.2f}s")
print(f"   Total time:          {extraction_result['total_seconds']:.2f}s")
print(f"")
print(f"🚀 Performance:")
print(f"   Throughput:          {extraction_result['throughput_per_hour']:.0f} images/hour")
avg_per_image = extraction_result['total_seconds'] / extraction_result['total_images'] if extraction_result['total_images'] > 0 else 0
print(f"   Avg per image:       {avg_per_image:.2f}s")
print("=" * 60)


## Step 5: View Output Files

List the cropped product images in the output stage.


In [ ]:
# Refresh stage directory metadata
session.sql("ALTER STAGE AD_OUTPUT_STAGE REFRESH").collect()

# List output files
output_df = session.sql("""
    SELECT 
        RELATIVE_PATH,
        SIZE AS file_size_bytes
    FROM DIRECTORY(@AD_OUTPUT_STAGE)
    WHERE RELATIVE_PATH LIKE '%_product_%.png'
    ORDER BY RELATIVE_PATH
""")

output_count = output_df.count()
print(f"Found {output_count} product crops in @AD_OUTPUT_STAGE")
output_df.show(20)


## Step 6: Download Output Images (Optional)

Use SnowCLI to download the cropped product images to your local machine.

```bash
# Download all product crops
snow stage copy @SHALION_HF_DEMO.PRODUCT_EXTRACTION.AD_OUTPUT_STAGE/ ./output/ --overwrite

# Verify download
ls -la ./output/*.png | head -20
```

The output files are named: `{original_image}_product_{index}.png`

Example:
- `template_1_01_product_000.png` - First product from template_1_01.jpeg
- `template_1_01_product_001.png` - Second product from template_1_01.jpeg


## Next Steps

After extracting product crops, you can:

1. **Embed with Cortex**: Use `SNOWFLAKE.CORTEX.EMBED_TEXT_1024()` or image embedding
2. **Brand Classification**: Match embeddings against known brand references using Cortex Search
3. **Quality Review**: Download samples to verify extraction quality
4. **Scale Up**: Increase `MAX_NODES` in compute pool for higher throughput
